In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch import nn
import pathlib
from torch.utils.data import DataLoader
from torchvision import *

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
transformtrain = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])
transformtest = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

In [ ]:
trainds = datasets.ImageFolder('../input/waste-classification-data/DATASET/TRAIN/', transform=transformtrain)
testds = datasets.ImageFolder('../input/waste-classification-data/DATASET/TEST/', transform=transformtest)

In [ ]:
trainloader = DataLoader(trainds, batch_size=256, shuffle=True)
testloader = DataLoader(testds, batch_size=64, shuffle=False)

In [ ]:
root = pathlib.Path('../input/waste-classification-data/DATASET/TRAIN/')
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
model = models.vgg19(pretrained=True).to(device)
for param in model.features.parameters():
    param.requires_grad = False


In [ ]:
model

In [ ]:
model.classifier[6] = nn.Linear(model.classifier[6].in_features, len(classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.00001)

In [ ]:
trainlosses = []
testlosses = []
for e in range(50):
    trainloss = 0
    traintotal = 0
    trainsuccessful = 0
    for traininput, trainlabel in trainloader:
        traininputs, trainlabels = traininput.to(device), trainlabel.to(device)
        optimizer.zero_grad()
        trainpredictions = model(traininputs)
        _, trainpredict = torch.max(trainpredictions.data, 1)
        loss = criterion(trainpredictions, trainlabels)
        loss.backward()
        optimizer.step()
        trainloss += loss.item()
        traintotal += trainlabels.size(0)
        trainsuccessful += (trainpredict == trainlabels).sum().item()
    else:
        testloss = 0
        testtotal = 0
        testsuccessful = 0
        with torch.no_grad():
            for testinput, testlabel in testloader:
                testinputs, testlabels = testinput.to(device), testlabel.to(device)
                testpredictions = model(testinputs)
                _, testpredict = torch.max(testpredictions.data, 1)
                tloss = criterion(testpredictions, testlabels)
                testloss += tloss.item()
                testtotal += testlabels.size(0)
                testsuccessful += (testpredict == testlabels).sum().item()
        trainlosses.append(trainloss/len(trainloader))
        testlosses.append(testloss/len(testloader))
        print('Train Accuracy %{:.2f}'.format(100*trainsuccessful/traintotal))
        print('Test Accuracy %{:.2f}'.format(100*testsuccessful/testtotal))

In [ ]:
!ls ../input/waste-classification-data/DATASET/TEST/O/O_12568.jpg

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
!ls ../input/waste-classification-data/DATASET/TEST/O/O_12568.jpg

In [ ]:
img = Image.open('../input/waste-classification-data/DATASET/TEST/O/O_12568.jpg')

In [ ]:
nimg = np.array(img)

In [ ]:
plt.imshow(nimg)

In [ ]:
pimg = transformtest(img).unsqueeze(0).to(device)

In [ ]:
pimg.shape

In [ ]:
prediction = model(pimg)

In [ ]:
_, tpredict = torch.max(prediction.data, 1)

In [ ]:
classes[tpredict[0].item()]